In [5]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
f=open(r'templeSparseRing\templeSR_par.txt','r')
assert f is not None
n= int(f.readline())

#reading the information of the 2nd image 
l=f.readline().split()
im1_fn=l[0]

k1=np.array([float(i) for i in l[1:10]]).reshape((3,3))
R1=np.array([float(i) for i in l[10:19]]).reshape((3,3))
t1=np.array([float(i) for i in l[19:22]]).reshape((3,1))

#reading the information of the 2nd image 

l=f.readline().split()
im2_fn=l[0]

k2=np.array([float(i) for i in l[1:10]]).reshape((3,3))
R2=np.array([float(i) for i in l[10:19]]).reshape((3,3))
t2=np.array([float(i) for i in l[19:22]]).reshape((3,1))

#read the two images and show
im1= cv.imread(r'templeSparseRing/' + im1_fn,cv.IMREAD_COLOR)
im2= cv.imread(r'templeSparseRing/' + im2_fn,cv.IMREAD_COLOR)
assert im1 is not None

#compute p1 and p2
#P=k*[R|t]
P1=k1@np.hstack((R1,t1)) 
P2=k2@np.hstack((R2,t2))